### 단일 intent FewShotChatMessagePrompt 예제

In [171]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_chroma import Chroma
# from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings


chat = ChatOpenAI(
    model="gpt-3.5-turbo-0125", 
    temperature=0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)


examples = [
    {
        "intent": "재생",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        넷플릭스, 라디오, 미디어 통합검색, 실시간채널 검색/재생, 음악 검색/재생
        """,
    },
    {
        "intent": "교통",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        내비정보, 대중교통, 버스, 택시, 지하철
        """,
    },
    {
        "intent": "생활 정보",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        날씨정보 요청, 뉴스 브리핑, 뉴스 속보 요청, 환율 정보 요청, 프로야구
        """,
    },
    {
        "intent": "일상 대화",
        "answer": """
        관련 의도를 생성해본 입니다:
        가이드 발화, 감성채팅, 미분류, 미응대, 서비스 제안
        """,
    },
    {
        "intent": "일정",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        일정목록 입력, 일정목록 조회
        """,
    },
    {
        "intent": "메모",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        리마인더, 음성 메모, 음성 메시지
        """,
    },
    {
        "intent": "법률",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        법률상담, 법률정보, 법률서비스, 법률교육, 법률업무
        """,
    },
    {
        "intent": "법률 정보",
        "answer": """
        관련 의도를 생성해본 결과 입니다:
        법제처, 기본법령, 국가법령정보센터, 자치법규정보시스템, 한국법령정보원, 헌법재판소, 정부24, 입법 동향
        """,
    },
]


# SemanticSimilarityExampleSelector는 의미론적 의미에 따라 입력과 유사한 예제를 선택합니다.

to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

example_selector = SemanticSimilarityExampleSelector(
  vectorstore=vectorstore,
  k=1 # 생성할 예제 개수입니다.
)

# example_selector = SemanticSimilarityExampleSelector.from_examples(
#   examples,
#   OpenAIEmbeddings(),  # 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 데 사용되는 임베딩 클래스입니다.
#   Chroma,  # 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스입니다.
#   k=1 # 생성할 예제 개수입니다.
# )
#출처: https://revf.tistory.com/280

# example_selector 테스트 코드
# print(example_selector.select_examples({"intent": "한국 법률"}))


# example_intent_prompt = ChatPromptTemplate.from_messages([
#     ("human", "다음의 단어와 관련한 상세의도 단어를 생성해주세요. {intent}"),
#     ("ai", "{answer}")
# ])

example_intent_prompt = ChatPromptTemplate.from_messages([
    ("human", "{intent}"),
    ("ai", "{answer}")
])

# few_shot_prompt = FewShotChatMessagePromptTemplate(
#     example_prompt=example_intent_prompt,
#     examples=examples,
# )


few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector, 
    example_prompt=example_intent_prompt,
)

# print(few_shot_prompt.format())

# small_intent_prompt = ChatPromptTemplate.from_messages([
#     ("system", "당신은 단어의 성질을 정확하게 파악하여 카테고리를 세분화 하는 기능을 가진 시스템입니다."),
#     example_prompt,
#     ("human", "다음의 단어에 대한 성질을 분석해서 세부 카테고리를 분류해주세요. {intent}")
# ])

# small_intent_prompt = ChatPromptTemplate.from_messages([
#     ("system", "당신은 입력 단어의 성격을 정확하게 파악하여 관련된 상세의도 단어를 생성하는 기능을 가진 시스템입니다."),
#     few_shot_prompt,
#     ("human", "다음 단어의 성격을 파악하여 관련된 상세의도 단어를 5개를 생성해주세요. {intent}")
# ])

small_intent_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 입력 받은 단어를 해석하여 관련되어있는 추가 단어를 생성하는 전문가입니다. 관련된 단어를 최대 5개 생성해주세요. 입력 받은 단어가 예시로 학습한 정보와 유사성을 띄고 있으면 가장 유사도가 높은 예시의 답변을 그대로 반환해주세요."),
    few_shot_prompt,
    ("human", "{intent}")
])

# # few_shot_prompt 잘 안되는 프롬프트 예시
# small_intent_prompt = ChatPromptTemplate.from_messages([
#     ("system", "당신은 입력 받은 단어를 해석하여 관련되어있는 추가 단어를 생성하는 전문가입니다. 관련된 단어를 최대 5개 생성해주세요."),
#     few_shot_prompt,
#     ("human", "{intent}")
# ])

# # few_shot_prompt 및 example_prompt 적용 확인 코드
# small_intent_prompt.format(intent="한국 법률")

final_chain = small_intent_prompt | chat

final_chain.invoke({
    "intent": "메모"
})




리마인더, 메모장, 스케줄, 일정, 알림

AIMessage(content='리마인더, 메모장, 스케줄, 일정, 알림', response_metadata={'finish_reason': 'stop'}, id='run-edd61b29-f4a3-487c-87de-12d19309dd51-0')